## **Práctica 7: Deep learning 2 - Transferencia de aprendizaje en CNNs**

Como hemos visto en la práctica anterior, las redes de neuronas convolucionales o simplemente **redes convolucionales** (CNNs, del inglés *convolutional neural networks*), son un tipo de redes neuronales profundas que utilizan imágenes como datos de entrada para resolver una tarea de clasificación o regresión.

A continuación, veremos un ejemplo en el que utilizaremos una red pre-entrenada en ImageNet y realizaremos un ajuste de parámetros para el problema a resolver, utilizando la librería [TensorFlow](https://www.tensorflow.org/).

### **1. Preparar los datos de entrenamiento**

En esta práctica vamos a utilizar un conjunto para clasificación de imágenes denominado **Flowers recognition**, adaptado de la versión disponible en [Kaggle](https://www.kaggle.com/alxmamaev/flowers-recognition). Para ello, es necesario subir a Google Drive el fichero [`DL_flowers.zip`](https://drive.google.com/file/d/1pQYJJ9D8Ho9YoZGL0M-r2PA_8e2D3Gm4/view?usp=sharing) que contiene:

*   Una carpeta `images` con 4315 imágenes de flores correspondientes a cinco clases (*daisy*, *dandelion*, *rose*, *sunflower*, *tulip*).
*   Un fichero `labels.csv` con los nombres de todas las imágenes (columna *image_name*) y la clase a la que pertenece cada imagen (columna *class*).

In [ ]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/SSII/DL_flowers.zip'  # ruta al fichero comprimido

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = 'flowers/'
imgs_path = data_path + "images/"
labels_path = data_path + "labels.csv"

# Leer el fichero CSV con las etiquetas
labels = pd.read_csv(labels_path, dtype = {"class": "category"})

Mounted at /content/gdrive
Archive:  /content/gdrive/My Drive/SSII/DL_flowers.zip
   creating: flowers/
  inflating: flowers/labels.csv      
   creating: flowers/images/
  inflating: flowers/images/16209331331_343c899d38.jpg  
  inflating: flowers/images/6204049536_1ac4f09232_n.jpg  
  inflating: flowers/images/9965757055_ff01b5ee6f_n.jpg  
  inflating: flowers/images/5001848317_b33d17ab7a_n.jpg  
  inflating: flowers/images/18828283553_e46504ae38.jpg  
  inflating: flowers/images/18302701228_2b5790b199_n.jpg  
  inflating: flowers/images/461632542_0387557eff.jpg  
  inflating: flowers/images/14167534527_781ceb1b7a_n.jpg  
  inflating: flowers/images/5700466891_2bcb17fa68_n.jpg  
  inflating: flowers/images/14087792403_f34f37ba3b_m.jpg  
  inflating: flowers/images/8021568040_f891223c44_n.jpg  
  inflating: flowers/images/34718882165_68cdc9def9_n.jpg  
  inflating: flowers/images/5777669976_a205f61e5b.jpg  
  inflating: flowers/images/3446285408_4be9c0fded_m.jpg  
  inflating: flowers

A continuación, utilizaremos el método [train_test_split()](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), disponible en la librería `scikit-learn`, para dividir los datos en tres particiones: **entrenamiento, validación y test**. En este ejemplo, utilizaremos el 70% de las imágenes para entrenamiento, el 15% para validación y el 15% para test.

In [ ]:
from sklearn.model_selection import train_test_split

# Crear las tres particiones de datos: entrenamiento, validación y test
seed = 0
train_data, test_data = train_test_split(labels, test_size=0.3, random_state=seed)
dev_data, test_data = train_test_split(test_data, test_size=0.5, random_state=seed)

# Actualizar los índices de cada partición
train_data = train_data.reset_index(drop=True)
dev_data = dev_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

A continuación, utilizaremos [`ImageDataGenerator()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para generar los *batches* de las tres particiones.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preparar los datos utilizando normalización (rescale)
datagen = ImageDataGenerator(rescale=1./255)

# Especificar información dependiente del conjunto de datos
classes = ["0","1","2","3","4"]  # clases del problema
n_classes = len(classes)         # número de clases
img_width = img_height = 224     # dimensiones de la imagen
x_col = 'image_name'             # nombres de las columnas en el fichero CSV
y_col = 'class'

# Generar los batches con los datos para las tres particiones
batch_size = 128  # tamaño del batch
train_generator = datagen.flow_from_dataframe(dataframe=train_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                              class_mode="categorical", target_size=(img_width,img_height),
                                              batch_size=batch_size, classes=classes)
dev_generator = datagen.flow_from_dataframe(dataframe=dev_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                            class_mode="categorical", target_size=(img_width,img_height),
                                            batch_size=batch_size, classes=classes)
test_generator = datagen.flow_from_dataframe(dataframe=test_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                             class_mode="categorical", target_size=(img_width,img_height),
                                             batch_size=batch_size, classes=classes)

Found 3020 validated image filenames belonging to 5 classes.
Found 647 validated image filenames belonging to 5 classes.
Found 648 validated image filenames belonging to 5 classes.


### **2. Crear una CNN**

Una alternativa a crear una CNN y entrenarla desde cero es utilizar modelos pre-entrenados con grandes conjuntos de datos. Para ello, vamos a utilizar el modelo [InceptionV3](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3) pre-entrenado con [ImageNet](https://www.image-net.org/), una base de datos con más de 14 millones de imágenes correspondientes a más de 20 mil clases. En la librería `TensorFlow` puedes encontrar otros [modelos pre-entrenados](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

En primer lugar, cargaremos la base convolucional (`include_top=False`) de una InceptionV3 pre-entrenada con ImageNet. A continuación, le añadiremos unas capas completamente conectadas que nos permitan resolver nuestro problema de clasificación multi-clase (5 clases). Los parámetros de las nuevas capas se inicializarán aleatoriamente y se aprenderán durante el proceso de entrenamiento. Con respecto al resto de parámetros del modelo, permacerán fijos o *congelados* (`trainable=False`); es decir, los parámetros de las capas de la base convolucional serán los del modelo base pre-entrenado con ImageNet y no sufrirán ajustes durante el proceso de entrenamiento.

In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

def get_model():

  # Cargar la base convolucional del modelo InceptionV3 pre-entrenado en ImageNet
  base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

  # Ajustar los parámetros de las nuevas capas del modelo, dejando fijos los parámetros del resto de capas
  for layer in base_model.layers:
      layer.trainable = False   # por defecto, layer.trainable es True

  # Añadir nuevas capas a continuación de la base convolucional, para resolver la tarea de aprendizaje
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.25)(x)
  x = Dense(1024, activation='relu')(x)

  # Añadir una última capa completamente conectada con 5 neuronas (número de clases) para obtener la salida de la red
  predictions = Dense(n_classes, activation='softmax')(x)

  # Crear el modelo final e imprimir su representacion en modo texto
  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  return model

### **3. Entrenar una CNN**

Una vez definida la arquitectura de la CNN (la base convolucional de InceptionV3 + una capa oculta de 1024 neuronas + una capa de salida), el siguiente paso es configurar el modelo para el entrenamiento. Para ello utilizaremos el método [`compile()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile), siendo estos algunos de sus parámetros más relevantes:

* *optimizer*: nombre del optimizador (`Adam`, `RMSProp`, etc.) y tasa de aprendizaje (`learning_rate`). Ver [tf.keras.optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* *loss*: función de pérdida (`mean_squared_error`, `binary_crossentropy`, `categorical_crossentropy`, etc.). Ver [tf.keras.losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses).
* *metrics*: métricas que evaluará el modelo durante el entrenamiento y la validación (`accuracy`, etc.). Ver [tf.keras.metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).


In [ ]:
from tensorflow.keras import optimizers

# Crear el modelo
model = get_model()

# Configurar el proceso de aprendizaje
model.compile(loss='categorical_crossentropy',                 # función de pérdida para problemas de clasificación multi-clase
              optimizer=optimizers.AdamW(learning_rate=0.005),
              metrics=['accuracy'])


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_470 (Conv2D)            (None, 111, 111, 32  864         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_470 (Batch  (None, 111, 111, 32  96         ['conv2d_470[0][0]']             
 Normalization)                 )                                                           

A continuación, entrenaremos el modelo para buscar los parámetros que hagan mínima la función de pérdida. Para ello utilizaremos el método [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), que necesita que le suministremos los datos de entrenamiento y validación, el número de *epochs* y el tamaño de *batch* (para calcular el número de *steps_per_epoch*).

In [ ]:
# Entrenar el modelo con los datos preparados previamente
model.fit(train_generator,
          epochs=15,   # número de epochs
          verbose=2,  # muestra información del error al finalizar cada epoch
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=dev_generator,
          validation_steps=len(dev_data)/batch_size)

Epoch 1/15
23/23 - 19s - loss: 0.1415 - accuracy: 0.9487 - val_loss: 0.4184 - val_accuracy: 0.8609 - 19s/epoch - 816ms/step
Epoch 2/15
23/23 - 16s - loss: 0.1193 - accuracy: 0.9553 - val_loss: 0.4118 - val_accuracy: 0.8702 - 16s/epoch - 684ms/step
Epoch 3/15
23/23 - 16s - loss: 0.1229 - accuracy: 0.9513 - val_loss: 0.4151 - val_accuracy: 0.8779 - 16s/epoch - 683ms/step
Epoch 4/15
23/23 - 16s - loss: 0.1001 - accuracy: 0.9623 - val_loss: 0.4212 - val_accuracy: 0.8733 - 16s/epoch - 680ms/step
Epoch 5/15
23/23 - 16s - loss: 0.0860 - accuracy: 0.9732 - val_loss: 0.4931 - val_accuracy: 0.8686 - 16s/epoch - 679ms/step
Epoch 6/15
23/23 - 16s - loss: 0.0881 - accuracy: 0.9672 - val_loss: 0.4286 - val_accuracy: 0.8764 - 16s/epoch - 674ms/step
Epoch 7/15
23/23 - 17s - loss: 0.0795 - accuracy: 0.9755 - val_loss: 0.4357 - val_accuracy: 0.8841 - 17s/epoch - 700ms/step
Epoch 8/15
23/23 - 16s - loss: 0.0784 - accuracy: 0.9715 - val_loss: 0.4700 - val_accuracy: 0.8702 - 16s/epoch - 679ms/step
Epoch 9/

### **4. Evaluar una CNN**

Hemos visto cómo crear y entrenar una CNN a partir de un modelo pre-entrenado, utilizando una configuración de hiperparámetros que no necesariamente es la mejor. Lo ideal sería realizar una búsqueda de hiperparámetros y, una vez obtenida la mejor configuración, evaluar el modelo sobre el conjunto de test y así obtener el resultado final.

A continuación, se muestra el código para evaluar el modelo final en el conjunto de test. Para ello utilizaremos el método [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) que necesita que le suministremos los datos de test y el tamaño del *batch* (para calcular el número de `steps`).

In [ ]:
# Evaluar el modelo en el conjunto de test
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_data)/batch_size, verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

5/5 [==============================] - 4s 640ms/step - loss: 0.6234 - accuracy: 0.8549
test_loss: 0.6234, test_acc: 0.8549


Por último, además de analizar el error obtenido, podemos utilizar el modelo entrenado para hacer predicciones. Para ello utilizaremos el método [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict), al que le suministraremos los datos sobre los que realizar las predicciones (en este ejemplo, los datos de test).

In [ ]:
import numpy as np

# Obtener las predicciones para todos los ejemplos del conjunto de test
predictions = model.predict(test_generator, verbose=1)

# Imprimir la predicción para los dos primeros ejemplos (los valores obtenidos representan las probabilidades para cada una de las 5 clases)
for i in range(0,2):
  print("\n Ejemplo", i)
  print("\t Probabilidades para las 5 clases:", predictions[i])
  print("\t Clase predicha: %i, Probabilidad: %.4f" % (np.argmax([predictions[i]]), np.max(predictions[i])))

6/6 [==============================] - 3s 457ms/step

 Ejemplo 0
	 Probabilidades para las 5 clases: [1.5659738e-04 9.9859601e-01 5.5620632e-07 1.2464238e-03 4.4701596e-07]
	 Clase predicha: 1, Probabilidad: 0.9986

 Ejemplo 1
	 Probabilidades para las 5 clases: [3.9250252e-04 1.2257758e-05 3.0080109e-05 5.0268336e-03 9.9453837e-01]
	 Clase predicha: 4, Probabilidad: 0.9945


### **5. Pruebas**

Realiza una serie de pruebas con diferentes configuraciones de hiperparámetros (tasa de aprendizaje, optimizador) e intenta mejorar los resultados. Prueba también a modificar la arquitectura utilizando otras capas en la parte final del modelo (*top*) e incluso cargando otras [CNNs pre-entrenadas con ImageNet](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

En lugar de que el modelo solamente aprenda los parámetros de las capas nuevas, es posible hacer *fine-tuning* de los parámetros de todas las capas (o bien seleccionar algunas concretas). Para ello basta modificar el parámetro *trainable* de las diferentes capas.

También puedes aplicar técnicas de *data augmentation*, modificando los parámetros del método [`ImageDataGenerator()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator); y técnicas para prevenir el sobreajuste utilizando, por ejemplo, la [capa dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout).